## Summary stats of planets
for purpose of prgramatically generating planets

[List of Solar System objects](https://en.wikipedia.org/wiki/List_of_Solar_System_objects_by_size#:~:text=Larger%20than%20400%20km%20%20%20%20Body,%20%202004%20%2013%20more%20rows%20)

In [1]:
import pandas as pd
import numpy as np


In [93]:
df = pd.read_excel('../data/Solar System Data.xlsx',engine='openpyxl',skiprows=1).drop(0)
df['radius'] = df['Radius.1'].apply(float)
df['mass'] = df['Mass.1'].str.replace(',','').apply(float)
df.head()

,Body,Image,Radius,Radius.1,Volume,Volume.1,Mass,Mass.1,Density,Gravity,Gravity.1,Type,# (R/M),Discovery,radius,mass
1,Sun,NaN,695508 ± ?,109.2,"1,409,300,000","1,301,000",1989100000,"333,000",1.409,274.0,27.94,G2V-class star,1,-,109.200,333000.000
2,Jupiter,NaN,69911±6,10.97,"1,431,280","1,321",1898187±88,317.83,1.3262±0.0003,24.79,2.528,gas giant planet; has rings,2,-,10.970,317.830
3,Saturn,NaN,58232±6_x000D_\n(136775 for main rings),9.140,"827,130",764,568317±13,95.162,0.6871±0.0002,10.44,1.065,gas giant planet; has rings,3,-,9.140,95.162
4,Uranus,NaN,25362±7,3.981,"68,340",63.1,86813±4,14.536,1.270±0.001,8.87,0.886,ice giant planet; has rings,4/5,1781,3.981,14.536
5,Neptune,NaN,24622±19,3.865,"62,540",57.7,102413±5,17.147,1.638±0.004,11.15,1.137,ice giant planet; has rings,5/4,1846,3.865,17.147


filtering on planets

In [111]:
def get_type(x):
    if 'gas giant' in x:
        return "gas"
    if "ice giant" in x:
        return "ice"
    if "dwarf" in x:
        return "dwarf"
    if "terrestrial" in x:
        return "terrestrial"
    if "moon" in x:
        return "moon"
    else:
        return "no type"

In [117]:
df['Type.lower'] = df['Type'].apply(lambda x: str(x).lower())
df['type'] = df['Type.lower'].apply(get_type)
df['is_planet'] = df['Type.lower'].str.contains('planet')
df['is_moon'] = df['Type.lower'].str.contains('moon')
pla = df.loc[df['is_planet']]
pla[['Body','Type','type']]

,Body,Type,type
2,Jupiter,gas giant planet; has rings,gas
3,Saturn,gas giant planet; has rings,gas
4,Uranus,ice giant planet; has rings,ice
5,Neptune,ice giant planet; has rings,ice
6,Earth,terrestrial planet,terrestrial
7,Venus,terrestrial planet,terrestrial
8,Mars,terrestrial planet,terrestrial
11,Mercury,terrestrial planet,terrestrial
17,Pluto_x000D_\n134340,dwarf planet; plutino; multiple,dwarf
18,Eris_x000D_\n136199,dwarf planet; SDO; binary,dwarf


In [118]:
ptypes = pd.DataFrame(pla.groupby('type').count()['Body'])
ptypes.columns = ['count']
ptypes['prob'] = ptypes['count']/14
ptypes['mass_mean'] = pd.Series(pla.groupby('type').mean()['mass'])
ptypes['mass_std'] = pd.Series(pla.groupby('type').std()['mass'])
ptypes['radius_mean'] = pd.Series(pla.groupby('type').mean()['radius'])
ptypes['radius_std'] = pd.Series(pla.groupby('type').std()['radius'])
ptypes

,count,prob,mass_mean,mass_std,radius_mean,radius_std
type,,,,,,
dwarf,6,0.428571,0.001106,0.001110,0.1290,0.045917
gas,2,0.142857,206.496000,157.450053,10.0550,1.294005
ice,2,0.142857,15.841500,1.846256,3.9230,0.082024
terrestrial,4,0.285714,0.494325,0.483495,0.7162,0.305601


In [119]:
ptypes.T.to_dict()

{'dwarf': {'count': 6.0,
  'prob': 0.42857142857142855,
  'mass_mean': 0.0011061666666666668,
  'mass_std': 0.0011102811205576121,
  'radius_mean': 0.129,
  'radius_std': 0.04591683787021923},
 'gas': {'count': 2.0,
  'prob': 0.14285714285714285,
  'mass_mean': 206.49599999999998,
  'mass_std': 157.45005275324613,
  'radius_mean': 10.055,
  'radius_std': 1.2940054095713813},
 'ice': {'count': 2.0,
  'prob': 0.14285714285714285,
  'mass_mean': 15.8415,
  'mass_std': 1.846255805678074,
  'radius_mean': 3.923,
  'radius_std': 0.08202438661763928},
 'terrestrial': {'count': 4.0,
  'prob': 0.2857142857142857,
  'mass_mean': 0.49432499999999996,
  'mass_std': 0.4834953524423305,
  'radius_mean': 0.7162,
  'radius_std': 0.3056014070648236}}

In [120]:
mon = df.loc[df['is_moon']]
mon[['Body','Type','type']]

,Body,Type,type
9,Ganymede_x000D_\nJupiter III,moon of Jupiter (icy),moon
10,Titan_x000D_\nSaturn VI,moon of Saturn (icy),moon
12,Callisto_x000D_\nJupiter IV,moon of Jupiter (icy),moon
13,Io_x000D_\nJupiter I,moon of Jupiter (terrestrial),terrestrial
14,Moon (Luna)_x000D_\nEarth I,moon of Earth (terrestrial),terrestrial
15,Europa_x000D_\nJupiter II,moon of Jupiter (terrestrial),terrestrial
16,Triton_x000D_\nNeptune I,moon of Neptune (icy),moon
19,Titania_x000D_\nUranus III,moon of Uranus,moon
21,Rhea_x000D_\nSaturn V,moon of Saturn,moon
22,Oberon_x000D_\nUranus IV,moon of Uranus,moon


In [122]:
mtypes = pd.DataFrame(mon.groupby('type').count()['Body'])
mtypes.columns = ['count']
mtypes['prob'] = mtypes['count']/len(mon)
mtypes['mass_mean'] = pd.Series(mon.groupby('type').mean()['mass'])
mtypes['mass_std'] = pd.Series(mon.groupby('type').std()['mass'])
mtypes['radius_mean'] = pd.Series(mon.groupby('type').mean()['radius'])
mtypes['radius_std'] = pd.Series(mon.groupby('type').std()['radius'])
mtypes

,count,prob,mass_mean,mass_std,radius_mean,radius_std
type,,,,,,
moon,13,0.8125,0.005513,0.009417,0.179662,0.129185
terrestrial,3,0.1875,0.011778,0.003512,0.267867,0.020874


In [124]:
mtypes.T.to_dict()

{'moon': {'count': 13.0,
  'prob': 0.8125,
  'mass_mean': 0.005513153846153847,
  'mass_std': 0.009417033882333949,
  'radius_mean': 0.17966153846153846,
  'radius_std': 0.12918538706358867},
 'terrestrial': {'count': 3.0,
  'prob': 0.1875,
  'mass_mean': 0.011778333333333333,
  'mass_std': 0.0035116816959020257,
  'radius_mean': 0.26786666666666664,
  'radius_std': 0.020873987001369262}}